In [1]:
from config import *
from utilities import get_model_name
from dataset import ClassificationDataset

import functools
import numpy as np
import pandas as pd

results_dict = {}
internal, external = 11, 12
encode_method = "dummy"
impute_method = "itbagdt" #TODO
fs_method, fs_ratio = "dnp", 0.5  #dnp & graces
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

Number of Selected Features: 100%|██████████| 71/71 [06:09<00:00,  5.21s/it]


In [2]:
# from sklearn.neural_network import MLPClassifier
from utilities import MLPClassifier

model_fn = functools.partial(
    MLPClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    # hidden_layer_sizes = (64, 32)
    # config       lr_range            worst_auc
    # adam w/ bn:  [2e-5, 5e-5, 1e-4]  0.8942
    # adam w/o bn: [5e-5, 1e-4, 2e-4]  0.9148
    # sgdm w/ bn:  [1e-3, 2e-3, 5e-3]  0.8982
    # sgdm w/o bn: [5e-3, 1e-2, 2e-2]  0.9177

    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

MLPClassifier


Epoch 1/50:  50%|█████     | 13/26 [00:00<00:00, 126.44batch/s, loss=0.287]

model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9165 | AUPR: 0.6518 | Acc: 95.35%


model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9195 | AUPR: 0.6583 | Acc: 95.33%


model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9223 | AUPR: 0.6665 | Acc: 95.32%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9174 | AUPR: 0.6530 | Acc: 95.34%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9202 | AUPR: 0.6582 | Acc: 95.32%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9216 | AUPR: 0.6626 | Acc: 95.36%
best model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9223 | AUPR: 0.6665 | Acc: 95.32%
Internal test: AUC: 0.9295 | AUPR: 0.6954 | Acc: 94.94%
External test: AUC: 0.9215 | AUPR: 0.6625 | Acc: 95.07%


In [3]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, probability=True,
    class_weight='balanced')
param_grid_list = [{
    'kernel': ['linear'],
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

SVC
model(kernel=linear, C=0.01)
Valid:         AUC: 0.9177 | AUPR: 0.6284 | Acc: 95.09%
model(kernel=linear, C=0.1)
Valid:         AUC: 0.9186 | AUPR: 0.6257 | Acc: 95.01%
model(kernel=linear, C=1)
Valid:         AUC: 0.9174 | AUPR: 0.6230 | Acc: 94.98%
best model(kernel=linear, C=0.1)
Valid:         AUC: 0.9186 | AUPR: 0.6257 | Acc: 95.01%
Internal test: AUC: 0.9248 | AUPR: 0.6733 | Acc: 94.89%
External test: AUC: 0.9158 | AUPR: 0.6142 | Acc: 94.60%


In [4]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto', p=2)
param_grid_list = [{
    'n_neighbors': [3, 5, 7],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

KNeighborsClassifier
model(n_neighbors=3)
Valid:         AUC: 0.7540 | AUPR: 0.4206 | Acc: 94.71%
model(n_neighbors=5)
Valid:         AUC: 0.7806 | AUPR: 0.4677 | Acc: 95.04%
model(n_neighbors=7)
Valid:         AUC: 0.8034 | AUPR: 0.4998 | Acc: 95.07%
best model(n_neighbors=7)
Valid:         AUC: 0.8034 | AUPR: 0.4998 | Acc: 95.07%
Internal test: AUC: 0.8079 | AUPR: 0.5388 | Acc: 94.96%
External test: AUC: 0.8106 | AUPR: 0.5132 | Acc: 94.81%


In [5]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GaussianNB
model()
Valid:         AUC: 0.8768 | AUPR: 0.4232 | Acc: 84.04%
best model()
Valid:         AUC: 0.8768 | AUPR: 0.4232 | Acc: 84.04%
Internal test: AUC: 0.8835 | AUPR: 0.4512 | Acc: 84.31%
External test: AUC: 0.8747 | AUPR: 0.4006 | Acc: 82.72%


In [6]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

LogisticRegression


model(C=1.0)
Valid:         AUC: 0.9192 | AUPR: 0.6500 | Acc: 95.30%
model(C=10.0)
Valid:         AUC: 0.9188 | AUPR: 0.6492 | Acc: 95.25%
model(C=100.0)
Valid:         AUC: 0.9181 | AUPR: 0.6486 | Acc: 95.26%
best model(C=1.0)
Valid:         AUC: 0.9192 | AUPR: 0.6500 | Acc: 95.30%
Internal test: AUC: 0.9267 | AUPR: 0.6861 | Acc: 95.11%
External test: AUC: 0.9150 | AUPR: 0.6328 | Acc: 94.87%


In [7]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

DecisionTreeClassifier
model(criterion=gini)
Valid:         AUC: 0.8629 | AUPR: 0.5475 | Acc: 83.71%
model(criterion=log_loss)
Valid:         AUC: 0.8704 | AUPR: 0.5611 | Acc: 83.72%
model(criterion=entropy)
Valid:         AUC: 0.8704 | AUPR: 0.5611 | Acc: 83.72%
best model(criterion=log_loss)
Valid:         AUC: 0.8704 | AUPR: 0.5611 | Acc: 83.72%
Internal test: AUC: 0.8563 | AUPR: 0.5813 | Acc: 83.70%
External test: AUC: 0.8654 | AUPR: 0.5476 | Acc: 84.19%


In [8]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

RandomForestClassifier
model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9142 | AUPR: 0.6536 | Acc: 89.65%
model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9147 | AUPR: 0.6557 | Acc: 89.80%
model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9155 | AUPR: 0.6577 | Acc: 89.85%
model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.9156 | AUPR: 0.6573 | Acc: 89.91%
model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9154 | AUPR: 0.6578 | Acc: 90.13%
model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9159 | AUPR: 0.6585 | Acc: 90.13%
model(criterion=entropy, n_estimators=50)
Valid:         AUC: 0.9156 | AUPR: 0.6573 | Acc: 89.91%
model(criterion=entropy, n_estimators=100)
Valid:         AUC: 0.9154 | AUPR: 0.6578 | Acc: 90.13%
model(criterion=entropy, n_estimators=200)
Valid:         AUC: 0.9159 | AUPR: 0.6585 | Acc: 90.13%
best model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9159 | AUPR: 0.6585 | Ac

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

model = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GradientBoostingClassifier


model(criterion=friedman_mse)
Valid:         AUC: 0.9168 | AUPR: 0.6431 | Acc: 95.22%
model(criterion=squared_error)
Valid:         AUC: 0.9169 | AUPR: 0.6402 | Acc: 95.16%
best model(criterion=squared_error)
Valid:         AUC: 0.9169 | AUPR: 0.6402 | Acc: 95.16%
Internal test: AUC: 0.9264 | AUPR: 0.6897 | Acc: 95.28%
External test: AUC: 0.9205 | AUPR: 0.6517 | Acc: 95.11%


In [10]:
from sklearn.ensemble import AdaBoostClassifier
model = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

AdaBoostClassifier


model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9156 | AUPR: 0.6308 | Acc: 95.04%
model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9160 | AUPR: 0.6330 | Acc: 95.07%
model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9150 | AUPR: 0.6316 | Acc: 95.15%
best model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9160 | AUPR: 0.6330 | Acc: 95.07%
Internal test: AUC: 0.9190 | AUPR: 0.6736 | Acc: 94.99%
External test: AUC: 0.9155 | AUPR: 0.6165 | Acc: 94.73%


In [11]:
dataframe = pd.DataFrame(results_dict).transpose().reset_index()
dataframe.columns = ['model', 'val_auc', 'val_aupr', 'val_acc', 'intest_auc', 'intest_aupr', 'intest_acc', 'extest_auc', 'extest_aupr', 'extest_acc']
# dataframe.to_csv()
filename = os.path.join(RESULTS_DIR, encode_method, impute_method, f'{fs_method}{int(fs_ratio * 142)}.csv')
dirname = os.path.dirname(filename)
if not os.path.exists(dirname):
    os.makedirs(dirname)
dataframe.to_csv(filename, index=False)